In [ ]:
import requests
import pandas as pd
import numpy as np
from joblib import load
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta

model = load("/content/drive/MyDrive/data/Tennis-Prediction.joblib")

API_KEY = "bbad081e02e2214b97aa19f1f9151e0e6b84b0caf1e24a5cff55ff663c18382b"
BASE_URL = "https://api.api-tennis.com/tennis/"
now = datetime.now()

date_start = now.strftime('%Y-%m-%d')
date_stop = (now + timedelta(days=2)).strftime('%Y-%m-%d')
print(f"📦 Loading matches from {date_start} to {date_stop}")


# ---------------- Fetch Fixtures ----------------
fixture_params = {
    "method": "get_fixtures",
    "APIkey": API_KEY,
    "date_start": date_start,
    "date_stop": date_stop,
    "timezone": "Europe/London",
    "status": "Not Started"
}
fixture_response = requests.get(BASE_URL, params=fixture_params)
fixture_data = fixture_response.json().get("result", [])

matches_info = {}
for m in fixture_data:
    if m.get("event_status") == "" and m.get("event_final_result") == "-":
        match_id = m["event_key"]
        matches_info[str(match_id)] = {
            "Date": m["event_date"],
            "Time": m["event_time"],
            "Player_1": m["event_first_player"],
            "Player_2": m["event_second_player"],
            "Match": f"{m['event_first_player']} vs {m['event_second_player']}",
            "Level": m["event_type_type"]
        }

# ---------------- Fetch Odds ----------------
odds_result = {}
for i in range(2):  # today + tomorrow
    odds_date = (now + timedelta(days=i)).strftime('%Y-%m-%d')
    print(f"🎲 Fetching odds for {odds_date} ...")
    odds_params = {
        'method': 'get_odds',
        'APIkey': API_KEY,
        'date_start': odds_date,
        'date_stop': odds_date
    }
    try:
        odds_response = requests.get(BASE_URL, params=odds_params)
        if odds_response.status_code == 200:
            odds_data = odds_response.json()
            if isinstance(odds_data, dict) and odds_data.get('success') == 1:
                day_odds = odds_data.get('result', {})
                for match_id in day_odds:
                    if match_id in matches_info:
                        odds = day_odds[match_id].get("Home/Away", {})
                        home_odds = [float(v) for v in odds.get("Home", {}).values() if v]
                        away_odds = [float(v) for v in odds.get("Away", {}).values() if v]

                        if home_odds and away_odds:
                            avg_home = np.mean(home_odds)
                            avg_away = np.mean(away_odds)
                            match_data = matches_info[match_id]
                            match_data["Odds"] = round(min(avg_home, avg_away), 2)
                            match_data["Fav_Player"] = match_data["Player_1"] if avg_home < avg_away else match_data["Player_2"]
                            match_data["Is_Player1_Fav"] = avg_home < avg_away
                            odds_result[match_id] = match_data
        else:
            print(f"❌ Failed to load odds for {odds_date}")
    except Exception as e:
        print(f"⚠️ Error while loading odds for {odds_date}: {str(e)}")

# ---------------- DataFrame & Model ----------------
if not odds_result:
    print("⚠️ No matches with available odds.")
else:
    df = pd.DataFrame.from_dict(odds_result, orient='index')
    df["Level"] = LabelEncoder().fit_transform(df["Level"])
    df["log_odds"] = np.log(df["Odds"])
    df["inverse_odds"] = 1 / df["Odds"]
    df["prob_diff"] = 0
    df["risk_index"] = 0

    features = ['Odds', 'Level', 'log_odds', 'inverse_odds', 'prob_diff', 'risk_index']
    df["model_probability"] = model.predict_proba(df[features])[:, 1]
    df["model_expected_value"] = (df["model_probability"] * (df["Odds"] - 1)) - (1 - df["model_probability"])
    df["model_expected_value_percent"] = df["model_expected_value"] * 100
    df["prob_diff"] = df["model_probability"] - df["inverse_odds"]
    df["risk_index"] = (df["model_probability"] * df["Odds"]) - 1

    df["prediction"] = np.where(
        df["model_probability"] >= 0.5,
        df["Player_1"] + " to win match",
        df["Player_2"] + " to win match"
    )

    # ---------------- Filter High Value Bets ----------------
    value_bets = df[
        (df["model_probability"] >= 0.6) &
        (df["model_expected_value"] >= 0.06)
    ].copy()

    print("\n📅 High-Value Upcoming Matches:")
    if value_bets.empty:
        print("⚠️ No high-value bets found at the moment.")
    else:
        for _, row in value_bets.iterrows():
            print(f"🗓 {row['Date']} | {row['Match']}")
            print(f"🎯 Prediction: {row['prediction']}")
            print(f"📈 Prob: {row['model_probability']:.2%} | EV: {row['model_expected_value_percent']:.2f}% | Odds: {row['Odds']:.2f}")
            print("—" * 40)

        print(f"\n✅ Number of high-value bets found: {len(value_bets)}")


📦 Loading matches from 2025-06-26 to 2025-06-28
🎲 Fetching odds for 2025-06-26 ...
🎲 Fetching odds for 2025-06-27 ...

📅 High-Value Upcoming Matches:
🗓 2025-06-26 | I. Shinikova vs F. Jorge
🎯 Prediction: I. Shinikova to win match
📈 Prob: 91.88% | EV: 6.58% | Odds: 1.16
————————————————————————————————————————
🗓 2025-06-26 | E. Alexandrova vs I. Swiatek
🎯 Prediction: E. Alexandrova to win match
📈 Prob: 64.63% | EV: 7.93% | Odds: 1.67
————————————————————————————————————————
🗓 2025-06-26 | S. Sierra vs T. Gibson
🎯 Prediction: S. Sierra to win match
📈 Prob: 64.63% | EV: 9.22% | Odds: 1.69
————————————————————————————————————————
🗓 2025-06-26 | T. Lamasine vs A. Bakshi
🎯 Prediction: T. Lamasine to win match
📈 Prob: 90.21% | EV: 6.45% | Odds: 1.18
————————————————————————————————————————
🗓 2025-06-26 | L. Tien vs C. Moutet
🎯 Prediction: L. Tien to win match
📈 Prob: 60.05% | EV: 6.88% | Odds: 1.78
————————————————————————————————————————
🗓 2025-06-26 | A. Bolt vs M. Landaluce
🎯 Prediction: A

In [2]:
import requests
import pandas as pd
import numpy as np
from joblib import load
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta

# Load model
model = load("/content/drive/MyDrive/data/Tennis-Prediction.joblib")

# API setup
API_KEY = "bbad081e02e2214b97aa19f1f9151e0e6b84b0caf1e24a5cff55ff663c18382b"
BASE_URL = "https://api.api-tennis.com/tennis/"
now = datetime.now()

date_start = now.strftime('%Y-%m-%d')
date_stop = (now + timedelta(days=2)).strftime('%Y-%m-%d')
print(f"📦 Loading matches from {date_start} to {date_stop}")

# ---------------- Fetch Fixtures ----------------
fixture_params = {
    "method": "get_fixtures",
    "APIkey": API_KEY,
    "date_start": date_start,
    "date_stop": date_stop,
    "timezone": "Europe/London",
    "status": "Not Started"
}
fixture_response = requests.get(BASE_URL, params=fixture_params)
fixture_data = fixture_response.json().get("result", [])

matches_info = {}
for m in fixture_data:
    if m.get("event_status") == "" and m.get("event_final_result") == "-":
        match_id = m["event_key"]
        matches_info[str(match_id)] = {
            "Date": m["event_date"],
            "Time": m["event_time"],
            "Player_1": m["event_first_player"],
            "Player_2": m["event_second_player"],
            "Match": f"{m['event_first_player']} vs {m['event_second_player']}",
            "Level": m["event_type_type"]
        }

# ---------------- Fetch Odds ----------------
odds_result = {}
for i in range(2):  # today + tomorrow
    odds_date = (now + timedelta(days=i)).strftime('%Y-%m-%d')
    print(f"🎲 Fetching odds for {odds_date} ...")
    odds_params = {
        'method': 'get_odds',
        'APIkey': API_KEY,
        'date_start': odds_date,
        'date_stop': odds_date
    }
    try:
        odds_response = requests.get(BASE_URL, params=odds_params)
        if odds_response.status_code == 200:
            odds_data = odds_response.json()
            if isinstance(odds_data, dict) and odds_data.get('success') == 1:
                day_odds = odds_data.get('result', {})
                for match_id in day_odds:
                    if match_id in matches_info:
                        odds = day_odds[match_id].get("Home/Away", {})
                        home_odds = [float(v) for v in odds.get("Home", {}).values() if v]
                        away_odds = [float(v) for v in odds.get("Away", {}).values() if v]

                        if home_odds and away_odds:
                            avg_home = np.mean(home_odds)
                            avg_away = np.mean(away_odds)
                            match_data = matches_info[match_id]
                            match_data["Odds"] = round(min(avg_home, avg_away), 2)
                            match_data["Fav_Player"] = match_data["Player_1"] if avg_home < avg_away else match_data["Player_2"]
                            match_data["Is_Player1_Fav"] = avg_home < avg_away
                            odds_result[match_id] = match_data
        else:
            print(f"❌ Failed to load odds for {odds_date}")
    except Exception as e:
        print(f"⚠️ Error while loading odds for {odds_date}: {str(e)}")

# ---------------- DataFrame & Model ----------------
if not odds_result:
    print("⚠️ No matches with available odds.")
else:
    df = pd.DataFrame.from_dict(odds_result, orient='index')
    df["Level"] = LabelEncoder().fit_transform(df["Level"])
    df["log_odds"] = np.log(df["Odds"])
    df["inverse_odds"] = 1 / df["Odds"]

    # Initial zero-fill
    df["prob_diff"] = 0
    df["risk_index"] = 0

    features = ['Odds', 'Level', 'log_odds', 'inverse_odds', 'prob_diff', 'risk_index']
    df["model_probability"] = model.predict_proba(df[features])[:, 1]
    df["model_expected_value"] = (df["model_probability"] * (df["Odds"] - 1)) - (1 - df["model_probability"])
    df["model_expected_value_percent"] = df["model_expected_value"] * 100
    df["prob_diff"] = df["model_probability"] - df["inverse_odds"]
    df["risk_index"] = (df["model_probability"] * df["Odds"]) - 1

    df["prediction"] = np.where(
        df["model_probability"] >= 0.5,
        df["Player_1"] + " to win match",
        df["Player_2"] + " to win match"
    )

    # ---------------- Filter High Value Bets ----------------
    value_bets = df[
    (df["model_probability"] >= 0.6) &
    (df["prob_diff"] >= 0.06)
    ].copy()



    print("\n📅 High-Value Upcoming Matches:")
    if value_bets.empty:
        print("⚠️ No high-value bets found at the moment.")
    else:
        for _, row in value_bets.iterrows():
            print(f"🗓 {row['Date']} | {row['Match']}")
            print(f"🎯 Prediction: {row['prediction']}")
            print(f"📈 Prob: {row['model_probability']:.2%} | EV: {row['model_expected_value_percent']:.2f}% | Odds: {row['Odds']:.2f}")
            print("—" * 40)

        print(f"\n✅ Number of high-value bets found: {len(value_bets)}")


📦 Loading matches from 2025-06-26 to 2025-06-28
🎲 Fetching odds for 2025-06-26 ...
🎲 Fetching odds for 2025-06-27 ...

📅 High-Value Upcoming Matches:
🗓 2025-06-26 | E. Seibold vs C. Van Zonneveld
🎯 Prediction: E. Seibold to win match
📈 Prob: 91.88% | EV: 7.49% | Odds: 1.17
————————————————————————————————————————
🗓 2025-06-26 | H. Giavara vs K. Chung
🎯 Prediction: H. Giavara to win match
📈 Prob: 91.88% | EV: 7.49% | Odds: 1.17
————————————————————————————————————————

✅ Number of high-value bets found: 2


In [15]:
output_path = "/content/drive/MyDrive/sav/high_value_upcoming_bets.csv"
value_bets[[
    "Date", "Match", "Odds", "model_probability",
    "model_expected_value_percent", "prediction"
]].to_csv(output_path, index=False)

print(f"\n📥 Results have been saved to file: {output_path}")




📥 Results have been saved to file: /content/drive/MyDrive/sav/high_value_upcoming_bets.csv


In [9]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/sav/Tennis Predictions Backtest.csv')
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

bet_amount = 10

df['Profit'] = df.apply(lambda row: (row['Odds'] * bet_amount - bet_amount) if row['Outcome'] == 'Won' else -bet_amount, axis=1)

daily_results = df.groupby(df['Date'].dt.date).agg(
    Win_Percentage=('Outcome', lambda x: (x == 'Won').mean() * 100),
    Daily_Profit=('Profit', 'sum')
).reset_index()

daily_results['Cumulative_Profit'] = daily_results['Daily_Profit'].cumsum()

overall_win_percentage = (df['Outcome'] == 'Won').mean() * 100
overall_profit = df['Profit'].sum()

daily_results.to_csv('/content/drive/MyDrive/sav/Tennis_Backtest_Results.csv', index=False)

print(daily_results)
print(f"\nOverall Win Percentage: {overall_win_percentage:.2f}%")
print(f"Overall Profit: ${overall_profit:.2f}")


           Date  Win_Percentage  Daily_Profit  Cumulative_Profit
0    2023-01-01       79.411765        105.96             105.96
1    2023-01-02       54.347826        -39.47              66.49
2    2023-01-03       61.904762         19.37              85.86
3    2023-01-04       38.461538       -100.70             -14.84
4    2023-01-05       64.444444         60.57              45.73
..          ...             ...           ...                ...
331  2025-11-04       60.869565        -16.52           20367.65
332  2025-12-01       63.888889        141.36           20509.01
333  2025-12-02       70.370370        156.71           20665.72
334  2025-12-03       64.285714         -7.39           20658.33
335  2025-12-04       61.904762        -24.33           20634.00

[336 rows x 4 columns]

Overall Win Percentage: 67.68%
Overall Profit: $51539.14
